## 1단계: TensorFlow의 기본 뼈대 이해하기

- 하위 주제 1: 텐서플로우란? (핵심 원리: 그래프와 이식성)
- 하위 주제 2: 텐서(Tensor)와 변수(Variable) (데이터의 기본 단위)
- 하위 주제 3: 자동 미분과 GradientTape (학습의 엔진)

## 2단계: 모델에 데이터 공급하기 (tf.data)

- 하위 주제 1: 효율적인 데이터 파이프라인의 필요성
- 하위 주제 2: tf.data.Dataset으로 데이터 불러오고 가공하기
- 하위 주제 3: 성능 최적화의 비밀: prefetch

## 3단계: 모델을 위한 재료 준비 (Feature Engineering)

- 하위 주제 1: 특성(Feature)이란? (수치형 vs 범주형)
- 하위 주제 2: 특성 열(Feature Column)의 마법 (원-핫 인코딩 & 임베딩)


### 1-1 TenserFlow?
방향성 비순환 그래프(DAG, Directed Acyclic Graph)
- Node: Operaion 에 해당
- Edge: 데이터의 흐름 - Node 를 잇는 선

**텐서(데이터)**가 그래프를 따라 흐르면서(Flow) 연산되는것

파이썬으로 설계도를 생성하면 스마트폰, 웹브라우저, ... 다른 기기에서 Runtime이 실행하기 때문에 이식성(Portability)이 좋다


### 활동: 미니 레시피 분석하기
TensorFlow로 `(3 + 5) * 2` 이라는 간단한 계산을 한다고 가정. 

1. 이 레시피에서 '재료를 손질하고 섞는' 것과 같은 실제 행동 단계(연산)에 해당하는 노드(Node)는 무엇일까요?
2. 그 행동 단계들 사이를 흘러 다니는 '중간 재료(데이터)'에 해당하는 에지(Edge)는 무엇일까요?
3. 이 레시피에 따라 요리를 마쳤을 때 나오는 최종 결과물, 16이라는 숫자는 TensorFlow에서 부르는 용어로 무엇일까요?

- 노드 (Node): 덧셈, 곱셈 같은 연산.
- 에지 (Edge): 연산과 연산을 잇는 데이터의 흐름/경로.
- 텐서 (Tensor): 그 경로를 따라 흐르는 모든 데이터 (숫자, 벡터, 행렬 등).

---


### 1-2 Tensor와 Variable

Tensor?
데이터의 N차원 배열

- Rank 0 Tensor: scalar - 단일 데이터
- Rank 1 Tensor: Vector - 1차원 배열 Ex) - [1, 2, 3]
- Rank 2 Tensor: matrix - 2차원 배열 Ex) - [[1, 2, 3], [4, 5, 6]]
- Rank 3 Tensor: - 2차원 배열의 집합
- Rank N Tensor: rank n - 1 텐서의 집합

이전 텐서가 쌓여 이후 텐서를 구성한다.

수학에서의 1, 2, 3`차원` 과는 다르게, `rank tensor`는 데이터에 접근하기 위한 인덱스의 갯수로 이해를 해야한다. 

- 벡터 -> data[i]
- 행렬 -> data[i][j] -> 인덱스 2개
- 3차원 텐서 -> data[i][j][k] -> 인덱스 3개

#### 텐서의 종류

1. Constant - 상수
- tf.constant 로 생성. 이후 값은 불변

2. Variable - 변수
- tf.variable로 생성. 값을 변경 가능
- **가중치(weights)**를 저장할때 사용





In [ ]:
import tensorflow as tf

# 1. 상수 (Constant): 한 번 정해지면 바꿀 수 없어요.
my_constant = tf.constant(10)
print(f"상수 텐서: {my_constant}")

# 2. 변수 (Variable): 값을 바꿀 수 있어요.
my_variable = tf.Variable(100.0)
print(f"변수 텐서 (업데이트 전): {my_variable.numpy()}")

# 변수의 값을 10만큼 더해서 업데이트합니다. (학습 과정과 유사)
my_variable.assign_add(10.0)
print(f"변수 텐서 (업데이트 후): {my_variable.numpy()}")

상수 텐서: 10
변수 텐서 (업데이트 전): 100.0
변수 텐서 (업데이트 후): 110.0
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=110.0>


---

## 1-3 자동 미분과 GradientTape
모델의 예측이 정답과 떨어진 정도를 나타내는 **손실(Loss)** 을 최소화 하는 방향으로 가중치를 조정해야 한다.

**Loss** 가 가장 기울어 져 있는 **경사(Gradient)** 를 찾고, 가중치를 조정하는 과정을 **경사하강법(Gradient Descent)** 이라 한다.

**Loss** 가 기울어진 정도는 Loss함수를 미분하여 구할 수 있지만, 모든 가중치에 대해 손으로 계산하는 것은 불가능

`tf.GradientTape` 으로 효율적이고 쉽게 계산 가능하다.

- 녹화 시작: with tf.GradientTape() as tape: 라는 코드로 녹화기를 킨다.
- 연산 녹화: 녹화기가 켜진 동안, 모델이 예측값을 만들고 손실(loss)을 계산하는 모든 과정을 테이프에 차곡차곡 기록한다. 
- 테이프 되감기 (미분): 녹화가 끝나면, tape.gradient(loss, weights) 라는 명령으로 테이프를 거꾸로 되감는다. 이 과정에서 TensorFlow는 **연쇄 법칙(Chain Rule)**을  이용해 손실 값에 영향을 미친 모든 가중치의 그래디언트를 자동으로 계산

In [7]:
import tensorflow as tf

# 모델의 가중치(Variable)와 입력값, 정답을 준비합니다.
W = tf.Variable(3.0) # 현재 가중치
x = 2.0              # 입력 데이터
y_true = 10.0        # 실제 정답

# 1. 녹화 시작!
with tf.GradientTape() as tape:
  # 2. 녹화 중에 예측하고 손실을 계산합니다.
  y_pred = W * x   # 예측: 3.0 * 2.0 = 6.0
  loss = (y_pred - y_true)**2  # 손실: (6.0 - 10.0)^2 = 16.0

# 3. 녹화된 테이프를 이용해 그래디언트를 계산합니다.
# "loss(16.0)를 줄이려면 W(3.0)를 어느 방향으로 바꿔야 할까?"
grad_W = tape.gradient(loss, W)

print(f"현재 손실 값: {loss.numpy()}")
print(f"가중치 W에 대한 그래디언트: {grad_W.numpy()}")

현재 손실 값: 16.0
가중치 W에 대한 그래디언트: -16.0
